# Import Required Libraries
Import libraries such as NumPy, pandas, and Matplotlib for data manipulation and visualization.

In [ ]:
import boto3
s3 = boto3.client('s3')
print(s3.list_buckets())

# Load and Visualize Tennis Frame Data
Load the tennis frame data (e.g., ball positions) and visualize the y-coordinate trajectory of the ball.

In [ ]:
# Load the tennis frame data
# Assuming the data is stored in a CSV file named 'tennis_frame_data.csv'
# The CSV file should have columns like 'frame', 'x', 'y' representing the ball's position in each frame
data = pd.read_csv('tennis_frame_data.csv')

# Display the first few rows of the data to understand its structure
print(data.head())

# Visualize the y-coordinate trajectory of the ball
plt.figure(figsize=(10, 6))
plt.plot(data['frame'], data['y'], label='Ball Y-Coordinate', color='blue')
plt.title('Y-Coordinate Trajectory of the Ball')
plt.xlabel('Frame')
plt.ylabel('Y-Coordinate')
plt.legend()
plt.grid(True)
plt.show()

# Implement Original Hitting Point Logic
Implement the original logic that uses the y-coordinate of the ball to determine the hitting point.

In [ ]:
# Implement the original hitting point logic using the y-coordinate of the ball
# Define a function to determine the hitting point based on the y-coordinate
def determine_hitting_points(data):
    """
    Determine hitting points based on the y-coordinate of the ball.
    A hitting point is identified as a local maximum or minimum in the y-coordinate trajectory.
    
    Parameters:
        data (pd.DataFrame): DataFrame containing 'frame' and 'y' columns.
    
    Returns:
        pd.DataFrame: DataFrame with an additional 'hitting_point' column.
    """
    # Initialize a new column to store hitting point flags
    data['hitting_point'] = False

    # Iterate through the data to find local maxima or minima
    for i in range(1, len(data) - 1):
        if (data.loc[i, 'y'] > data.loc[i - 1, 'y'] and data.loc[i, 'y'] > data.loc[i + 1, 'y']) or \
           (data.loc[i, 'y'] < data.loc[i - 1, 'y'] and data.loc[i, 'y'] < data.loc[i + 1, 'y']):
            data.loc[i, 'hitting_point'] = True

    return data

# Apply the function to the dataset
data = determine_hitting_points(data)

# Visualize the hitting points on the y-coordinate trajectory
plt.figure(figsize=(10, 6))
plt.plot(data['frame'], data['y'], label='Ball Y-Coordinate', color='blue')
plt.scatter(data[data['hitting_point']]['frame'], data[data['hitting_point']]['y'], 
            color='red', label='Hitting Points', zorder=5)
plt.title('Hitting Points Based on Y-Coordinate')
plt.xlabel('Frame')
plt.ylabel('Y-Coordinate')
plt.legend()
plt.grid(True)
plt.show()

# Add Extremum-Based Hitting Point Logic
Enhance the logic by identifying the highest and lowest y-coordinates as potential hitting points.

In [ ]:
# Add Extremum-Based Hitting Point Logic

# Define a function to enhance the hitting point logic by considering extremum-based logic
def add_extremum_based_logic(data, frame_window=5):
    """
    Enhance the hitting point logic by identifying the highest and lowest y-coordinates
    within a fixed frame window as potential hitting points. Ensure only one hitting point
    per frame window to account for position inaccuracies.

    Parameters:
        data (pd.DataFrame): DataFrame containing 'frame', 'y', and 'hitting_point' columns.
        frame_window (int): Number of frames to consider for extremum detection.

    Returns:
        pd.DataFrame: DataFrame with updated 'hitting_point' column.
    """
    # Initialize a new column to store enhanced hitting point flags
    data['enhanced_hitting_point'] = False

    # Iterate through the data in steps of the frame window
    for start in range(0, len(data), frame_window):
        # Define the end of the current frame window
        end = min(start + frame_window, len(data))

        # Extract the subset of data for the current frame window
        window_data = data.iloc[start:end]

        # Identify the index of the maximum and minimum y-coordinates in the window
        max_idx = window_data['y'].idxmax()
        min_idx = window_data['y'].idxmin()

        # Mark the maximum and minimum as hitting points, ensuring only one per window
        data.loc[max_idx, 'enhanced_hitting_point'] = True
        data.loc[min_idx, 'enhanced_hitting_point'] = True

    return data

# Apply the enhanced logic to the dataset
data = add_extremum_based_logic(data)

# Visualize the enhanced hitting points on the y-coordinate trajectory
plt.figure(figsize=(10, 6))
plt.plot(data['frame'], data['y'], label='Ball Y-Coordinate', color='blue')
plt.scatter(data[data['enhanced_hitting_point']]['frame'], data[data['enhanced_hitting_point']]['y'], 
            color='green', label='Enhanced Hitting Points', zorder=5)
plt.title('Enhanced Hitting Points with Extremum-Based Logic')
plt.xlabel('Frame')
plt.ylabel('Y-Coordinate')
plt.legend()
plt.grid(True)
plt.show()

# Integrate Single Hitting Point Constraint
Ensure that only one hitting point is selected within a fixed number of frames to account for potential position errors.

In [ ]:
# Define a function to ensure only one hitting point is selected within a fixed frame window
def integrate_single_hitting_point_constraint(data, frame_window=5):
    """
    Ensure that only one hitting point is selected within a fixed frame window.
    This accounts for potential position errors and avoids multiple hitting points in the same window.

    Parameters:
        data (pd.DataFrame): DataFrame containing 'frame', 'y', and 'enhanced_hitting_point' columns.
        frame_window (int): Number of frames to consider for hitting point selection.

    Returns:
        pd.DataFrame: DataFrame with updated 'final_hitting_point' column.
    """
    # Initialize a new column to store the final hitting point flags
    data['final_hitting_point'] = False

    # Iterate through the data in steps of the frame window
    for start in range(0, len(data), frame_window):
        # Define the end of the current frame window
        end = min(start + frame_window, len(data))

        # Extract the subset of data for the current frame window
        window_data = data.iloc[start:end]

        # Filter the window data to include only rows marked as enhanced hitting points
        hitting_candidates = window_data[window_data['enhanced_hitting_point']]

        # If there are any hitting candidates, select the one with the highest y-coordinate
        if not hitting_candidates.empty:
            best_candidate_idx = hitting_candidates['y'].idxmax()
            data.loc[best_candidate_idx, 'final_hitting_point'] = True

    return data

# Apply the single hitting point constraint to the dataset
data = integrate_single_hitting_point_constraint(data)

# Visualize the final hitting points on the y-coordinate trajectory
plt.figure(figsize=(10, 6))
plt.plot(data['frame'], data['y'], label='Ball Y-Coordinate', color='blue')
plt.scatter(data[data['final_hitting_point']]['frame'], data[data['final_hitting_point']]['y'], 
            color='orange', label='Final Hitting Points', zorder=5)
plt.title('Final Hitting Points with Single Hitting Point Constraint')
plt.xlabel('Frame')
plt.ylabel('Y-Coordinate')
plt.legend()
plt.grid(True)
plt.show()

# Test and Compare the Optimized Logic
Test the optimized logic on sample data and compare its performance with the original logic.

In [ ]:
# Test and Compare the Optimized Logic

# Define a function to evaluate the performance of the original and optimized logic
def evaluate_hitting_point_logic(data):
    """
    Compare the performance of the original and optimized hitting point logic.

    Parameters:
        data (pd.DataFrame): DataFrame containing 'hitting_point', 'enhanced_hitting_point',
                             and 'final_hitting_point' columns.

    Returns:
        None
    """
    # Count the number of hitting points identified by each logic
    original_count = data['hitting_point'].sum()
    enhanced_count = data['enhanced_hitting_point'].sum()
    final_count = data['final_hitting_point'].sum()

    # Print the results
    print(f"Original Logic Hitting Points: {original_count}")
    print(f"Enhanced Logic Hitting Points: {enhanced_count}")
    print(f"Final Logic Hitting Points: {final_count}")

# Call the evaluation function
evaluate_hitting_point_logic(data)

# Visualize and compare the hitting points identified by each logic
plt.figure(figsize=(12, 8))
plt.plot(data['frame'], data['y'], label='Ball Y-Coordinate', color='blue')
plt.scatter(data[data['hitting_point']]['frame'], data[data['hitting_point']]['y'], 
            color='red', label='Original Hitting Points', zorder=5)
plt.scatter(data[data['enhanced_hitting_point']]['frame'], data[data['enhanced_hitting_point']]['y'], 
            color='green', label='Enhanced Hitting Points', zorder=5)
plt.scatter(data[data['final_hitting_point']]['frame'], data[data['final_hitting_point']]['y'], 
            color='orange', label='Final Hitting Points', zorder=5)
plt.title('Comparison of Hitting Point Logic')
plt.xlabel('Frame')
plt.ylabel('Y-Coordinate')
plt.legend()
plt.grid(True)
plt.show()